In [26]:
from math import exp
import pandas as pd
import numpy as np

In [27]:
gamma =0.84697

In [28]:

data = pd.read_excel("使用版2012-2023-2个相关因素_2位小数.xlsx")
data = data.iloc[0:, :]  # 这次去取全部数据i=1？
n = data.shape[0]
train_n = n - 2
X0 = data.iloc[:train_n, 1:].to_numpy()
X0

array([[ 3.17183149,  3.58265388,  4.06480061],
       [ 3.58572051,  3.75307967,  4.98606851],
       [ 4.06002553,  4.00334458,  5.64055044],
       [ 4.70435864,  4.23290831,  6.58427062],
       [ 5.24470556,  4.38322184,  7.52824642],
       [ 5.58501584,  4.98932814,  8.60685072],
       [ 5.92532552,  5.59543445,  9.68545501],
       [ 6.01324623,  5.69594003, 10.37427573],
       [ 6.61325944,  6.56204715, 12.495697  ],
       [ 7.98908223,  7.43895863, 15.48479039]])

In [29]:
# 一阶累加生成
X1= np.cumsum(X0, 0)
y1 = X1[:, 0] ** (1 - gamma)

Zy = (y1[1:] + y1[:-1]) / 2
Z2 = (X1[1:, 1] + X1[:-1, 1]) / 2
Z3 = (X1[1:, 2] + X1[:-1, 2]) / 2
# 根据一阶累加生成的第二三列相乘得到序列 pq， 进而得到 Z23
pq = X1[:, 1] * X1[:, 2]
Z23 = (pq[1:] + pq[:-1]) / 2

In [30]:
B = np.array([-Zy, Z2, Z3, Z23]).T
ones_column = np.ones((B.shape[0], 1))
B = (1-gamma)*np.hstack((B, ones_column))
A = y1[1:] - y1[:-1]
    # 共五个值，分别为a, b2, b3, b23, b1

hat_a = np.dot(np.dot(np.linalg.inv(np.dot(B.T, B)), B.T), A)
a, b2, b3, b23, b1 = hat_a
print(a, b2, b3, b23, b1 )

1.95209353100398 -0.1378146312701266 0.10218572212642196 -0.0003320202993943994 3.512057265760637


In [31]:
# 这里重新获取下原数据，以实现所有数据的预测
X0 = data.iloc[:, 1:].to_numpy()
X1= np.cumsum(X0, 0)
# 对第一列一阶累加生成求 1-gamma 次幂
y1 = X1[:, 0] ** (1 - gamma)
X0


array([[ 3.17183149,  3.58265388,  4.06480061],
       [ 3.58572051,  3.75307967,  4.98606851],
       [ 4.06002553,  4.00334458,  5.64055044],
       [ 4.70435864,  4.23290831,  6.58427062],
       [ 5.24470556,  4.38322184,  7.52824642],
       [ 5.58501584,  4.98932814,  8.60685072],
       [ 5.92532552,  5.59543445,  9.68545501],
       [ 6.01324623,  5.69594003, 10.37427573],
       [ 6.61325944,  6.56204715, 12.495697  ],
       [ 7.98908223,  7.43895863, 15.48479039],
       [ 8.95623639,  8.38116319, 18.0884032 ],
       [ 9.19408275,  9.45294856, 19.79115704]])

In [32]:
  # 根据 y1 求 Zy，再分别根据一阶累加生成的第二三列生成Z2、Z3
Zy = (y1[1:] + y1[:-1]) / 2
Z2 = (X1[1:, 1] + X1[:-1, 1]) / 2
Z3 = (X1[1:, 2] + X1[:-1, 2]) / 2

# 根据一阶累加生成的第二三列相乘得到序列 pq， 进而得到 Z23
pq = X1[:, 1] * X1[:, 2]
Z23 = (pq[1:] + pq[:-1]) / 2

hat_y1 = [y1[0]]
for i in range(n-1):
    hat_y1.append(
           (1 - gamma) / (1 + 0.5 * a * (1 - gamma)) * (b2 * Z2[i] + b3 * Z3[i]) +
           (1 - 0.5 * a * (1 - gamma)) / (1 + 0.5 * a * (1 - gamma)) * hat_y1[-1] +
           (1 - gamma) * b1 / (1 + 0.5 * a * (1 - gamma)) +
           (1 - gamma) / (1 + 0.5 * a * (1 - gamma)) * (b23 * Z23[i])
    )

hat_y1 = np.array(hat_y1)
hat_x1 = np.exp(np.log(hat_y1) / (1 - gamma))
hat_x0 = np.diff(hat_x1)
print(hat_x0)

[3.53074815 4.27330189 4.59863398 5.03524797 5.62918893 5.96575895
 6.18042467 6.60904409 7.89728317 9.82455401 9.05089324]


In [33]:
original_x1 = data.iloc[1:, 1]
APE = np.abs(hat_x0 - original_x1) / original_x1
MAPE=np.mean(APE)
print(APE)
print(MAPE)

1     0.015331
2     0.052531
3     0.022474
4     0.039937
5     0.007909
6     0.006824
7     0.027802
8     0.000637
9     0.011491
10    0.096951
11    0.015574
Name: 主营业务收入（亿元）, dtype: float64
0.027041855434919403


In [34]:
pd.DataFrame(hat_x0).to_excel('IENGBM_hatx0.xlsx')
pd.DataFrame(APE).to_excel('IENGBM_APE.xlsx')